# [6.4 Planning as a CSP](http://artint.info/2e/html/ArtInt2e.Ch6.S4.html)
- [Implementation Details](http://artint.info/AIPython/aipython.pdf#page=92) (page 92)

## About

In forward planning, the search is constrained by the initial state and only uses the goal as a stopping criterion and as a source for heuristics. In regression planning, the search is constrained by the goal and only uses the start state as a stopping criterion and as a source for heuristics. By converting the problem to a constraint satisfaction problem (CSP), the initial state can be used to prune what is not reachable and the goal to prune what is not useful. The CSP will be defined for a finite number of steps; the number of steps can be adjusted to find the shortest plan. We can apply one of the CSP methods we learnt in order to solve this CSP and thus find a plan.

## Instructions

Each section header contains a link to the corresponding chapter in the accompanying textbook, and an "Implementation Details" link provided throughout tells you how the implementation works. Before using this notebook, make sure you have followed the [installation instructions](https://aispace2.github.io/AISpace2/install.html) beforehand.

You can run each cell by selecting it and pressing *Ctrl+Enter*. Alternatively, you can click the *Play* button in the toolbar, to the left of the stop button. 

For more information, including how the code in this notebook differs from that in [AIPython](aipython.org), check out the [Reference](https://aispace2.github.io/AISpace2/reference.html).

In [ ]:
from aispace2.jupyter.csp import Displayable, visualize
from aipython.cspConsistency import Con_solver, select, partition_domain, copy_with_assign

class Con_solver(Displayable):
    def __init__(self, csp, **kwargs):
        self.csp = csp
        super().__init__(**kwargs)    # Or Displayable.__init__(self,**kwargs)

    @visualize
    def make_arc_consistent(self, orig_domains=None, to_do=None):
        """Makes this CSP arc-consistent using generalized arc consistency
        orig_domains is the original domains
        to_do is a set of (variable,constraint) pairs
        returns the reduced domains
        """
        if orig_domains is None:
            orig_domains = self.csp.domains
        if to_do is None:
            to_do = {(var, const) for const in self.csp.constraints
                     for var in const.scope}
        else:
            to_do = to_do.copy()  # use a copy of to_do
        domains = orig_domains.copy()
        self.display(2, "Performing AC with domains", domains)
        while to_do:
            var, const = self.choose_arc(to_do)
            self.display(3, "Processing arc (", var, ",", const, ")")
            other_vars = [ov for ov in const.scope if ov is not var]
            new_domain = {val for val in domains[var]
                          if self.any_holds(domains, const, {var: val}, other_vars, 0)}
            if new_domain != domains[var]:
                self.display(4, "Arc: (", var, ",", const, ") is inconsistent")
                self.display(3, "Domain pruned", "dom(", var, ") =",
                             new_domain, " due to ", const)
                domains[var] = new_domain
                add_to_do = self.new_to_do(var, const)
                to_do |= add_to_do      # set union
                self.display(
                    3, "  adding", add_to_do if add_to_do else "nothing", "to to_do.")
            self.display(4, "Arc: (", var, ",", const, ") now consistent")
        self.display(2, "AC done. Reduced domains", domains)
        return domains

    def choose_arc(self, to_do):
        """Selects the arc to be taken from the to_do list.

        to_do list should be directly modified, or the algorithm will not terminate."""
        return self.visualizer.wait_for_arc_selection(to_do)

    def new_to_do(self, var, const):
        """returns new elements to be added to to_do after assigning
        variable var in constraint const.
        """
        return {(nvar, nconst) for nconst in self.csp.var_to_const[var]
                if nconst != const
                for nvar in nconst.scope
                if nvar != var}

    def any_holds(self, domains, const, env, other_vars, ind):
        """returns True if Constraint const holds for an assignment
        that extends env with the variables in other_vars[ind:]
        env is a dictionary
        Warning: this has side effects and changes the elements of env
        """
        if ind == len(other_vars):
            return const.holds(env)
        else:
            var = other_vars[ind]
            for val in domains[var]:
                # env = dict_union(env,{var:val})  # no side effects!
                env[var] = val
                if self.any_holds(domains, const, env, other_vars, ind + 1):
                    return True
            return False

    @visualize
    def solve_one(self, domains=None, to_do=None):
        """return a solution to the current CSP or False if there are no solutions
        to_do is the list of arcs to check
        """
        if domains is None:
            domains = self.csp.domains
        new_domains = self.make_arc_consistent(domains, to_do)
        if any(len(new_domains[var]) == 0 for var in domains):
            return False
        elif all(len(new_domains[var]) == 1 for var in domains):
            self.display(2, "solution:", {var: select(
                new_domains[var]) for var in new_domains})
            return {var: select(new_domains[var]) for var in domains}
        else:
            var = select(x for x in self.csp.variables if len(
                new_domains[x]) > 1)
            if var:
                split = len(domains[var]) // 2
                # a nonempty proper subset
                dom1 = set(list(new_domains[var])[:split])
                # copy with dom(var)=dom1
                new_dom1 = copy_with_assign(new_domains, var, dom1)
                dom2 = domains[var] - dom1
                # copy with dom(var)=dom2
                new_dom2 = copy_with_assign(new_domains, var, dom2)
                self.display(3, "...splitting", var, "into", dom1, "and", dom2)
                to_do = self.new_to_do(var, None)
                return self.solve_one(new_dom1, to_do) or self.solve_one(new_dom2, to_do)

    @visualize
    def solve_all(self, domains=None, to_do=None):
        """return a solution to the current CSP or False if there are no solutions
        to_do is the list of arcs to check
        """
        if domains is None:
            domains = self.csp.domains
        new_domains = self.make_arc_consistent(domains, to_do)
        if any(len(new_domains[var]) == 0 for var in domains):
            return []
        elif all(len(new_domains[var]) == 1 for var in domains):
            self.display(2, "solution:", {var: select(
                new_domains[var]) for var in new_domains})
            return [{var: select(new_domains[var]) for var in domains}]
        else:
            var = select(x for x in self.csp.variables if len(
                new_domains[x]) > 1)
            if var:
                split = len(domains[var]) // 2
                # a nonempty proper subset
                dom1 = set(list(new_domains[var])[:split])
                # copy with dom(var)=dom1
                new_dom1 = copy_with_assign(new_domains, var, dom1)
                dom2 = domains[var] - dom1
                # copy with dom(var)=dom2
                new_dom2 = copy_with_assign(new_domains, var, dom2)
                self.display(3, "...splitting", var, "into", dom1, "and", dom2)
                to_do = self.new_to_do(var, None)
                return self.solve_all(new_dom1, to_do) + self.solve_all(new_dom2, to_do)

You can click on an arc to run `make_arc_consistent()` on that selection, or choose "Fine Step", "Step", or "Auto Arc-Consistency" to randomly select an arc to make consistent. After arc consistency is finished, you can click on a variable with more than one value in its domain to split it. It will then prompt you to choose the domain of the first split.

In [ ]:
from aipython.stripsCSPPlanner import CSP_from_STRIPS
from aipython.stripsProblem import simple_problem1, simple_problem2, blocks1, blocks2, blocks3
from IPython.display import display

horizon = 1
csp = CSP_from_STRIPS(problem0, horizon)
solver = Con_solver(csp)
solver.text_size = 15
display(solver)

solver.solve_one()